## Attempt Using Dummy Game


Seemingly we have Timberwolves Cavs Game in the 2017-2018 season that we can try making some predictions on 

In [53]:
# Headers
import numpy as np
import pandas as pd
import matplotlib as plt
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import playbyplay, playbyplayv2, playbyplayv3

Getting Data for Game

In [54]:
# pbp 3
pbp3 = playbyplayv3.PlayByPlayV3(game_id="0021700807")
pbp3.get_json()
d3 = pbp3.get_dict()
# game has pbp
game = d3['game']
# Only care about actions
actions = game['actions']
d = {}
for a in actions:
    for k, v in a.items():
        if k in d:
            d[k].append(v)
        else:
            d[k] = [v]
df = pd.DataFrame(d)
df

,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,yLegacy,...,scoreHome,scoreAway,pointsTotal,location,description,actionType,subType,videoAvailable,shotValue,actionId
0,2,PT12M00.00S,1,0,,0,,,0,0,...,0,0,0,,Start of 1st Period (8:11 PM EST),period,start,0,0,1
1,4,PT12M00.00S,1,1610612739,CLE,202684,Thompson,T. Thompson,0,0,...,,,0,h,Jump Ball Thompson vs. Towns: Tip to Gibson,Jump Ball,,1,0,2
2,7,PT11M43.00S,1,1610612750,MIN,203952,Wiggins,A. Wiggins,223,152,...,,,0,v,MISS Wiggins 27' 3PT Jump Shot,Missed Shot,Jump Shot,1,3,3
3,8,PT11M33.00S,1,1610612750,MIN,201959,Gibson,T. Gibson,0,0,...,,,0,v,Gibson REBOUND (Off:1 Def:0),Rebound,Unknown,1,0,4
4,9,PT11M33.00S,1,1610612750,MIN,201959,Gibson,T. Gibson,0,-6,...,,,0,v,MISS Gibson 1' Tip Layup Shot,Missed Shot,Tip Layup Shot,1,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,654,PT00M01.00S,5,1610612739,CLE,1626224,Osman,C. Osman,0,0,...,,,0,h,SUB: Thomas FOR Osman,Substitution,,0,0,468
468,655,PT00M01.00S,5,1610612750,MIN,2037,Crawford,J. Crawford,0,0,...,,,0,v,SUB: Gibson FOR Crawford,Substitution,,0,0,469
469,659,PT00M00.00S,5,1610612739,CLE,2544,James,L. James,47,171,...,140,138,278,h,James 18' Turnaround Fadeaway (37 PTS) (Green ...,Made Shot,Turnaround Fadeaway shot,1,2,470
470,661,PT00M00.00S,5,0,,0,,,0,0,...,140,138,278,,End of 1st OT (10:40 PM EST),period,end,1,0,471


# Data Cleaning and Engineering

In [55]:
df.to_string("test.txt")
print(df.dtypes)

actionNumber       int64
clock             object
period             int64
teamId             int64
teamTricode       object
personId           int64
playerName        object
playerNameI       object
xLegacy            int64
yLegacy            int64
shotDistance       int64
shotResult        object
isFieldGoal        int64
scoreHome         object
scoreAway         object
pointsTotal        int64
location          object
description       object
actionType        object
subType           object
videoAvailable     int64
shotValue          int64
actionId           int64
dtype: object


Want to add numerically encode some non-numeric columns

In [56]:
def time_remaining(time:str):
    # Time will be of format PTXXMYY.YYS
    # Unsure of what PT means but will ignore for now
    # Returns number of time left in quarter in seconds
    minute_range = (2, 4)
    seconds_range = (5, 10)
    minutes = int  (time[minute_range[0] : minute_range[1]])
    seconds = float(time[seconds_range[0]: seconds_range[1]])
    return (minutes * 60) + seconds

First we numerically encode the game clock to number of seconds left in the quarter

In [57]:
df['time_remaining_in_q'] = df['clock'].apply(time_remaining)

We then encode a field goal make as 1 and a miss as 0

In [58]:
df['fg_made'] = df['shotResult'].apply(func=lambda x: 1 if x and x == "Made" else 0)


We can fill both the score home and away by filling the previous value

In [60]:
df['home_points'] = df['scoreHome'].replace("", np.nan)
df['away_points'] = df['scoreAway'].replace("", np.nan)
df['home_points'] = df['home_points'].ffill()
df['away_points'] = df['away_points'].ffill()
df.to_string("test.txt")
